# 📌 Extração

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1️⃣ Carregar dados
url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json"
df = pd.read_json(url)

# Expandir colunas aninhadas
df_customer = pd.json_normalize(df["customer"])
df_phone = pd.json_normalize(df["phone"])
df_internet = pd.json_normalize(df["internet"])
df_account = pd.json_normalize(df["account"])

# Juntar tudo
df_expanded = pd.concat([
    df.drop(columns=["customer", "phone", "internet", "account"]),
    df_customer,
    df_phone,
    df_internet,
    df_account
], axis=1)

# Tratar valores ausentes
df_expanded.fillna({"Churn Reason": "Não informado"}, inplace=True)

# Padronizar nomes de colunas
df_expanded.columns = df_expanded.columns.str.lower().str.strip().str.replace(" ", "_", regex=True).str.replace("\\.", "_", regex=True)

print("✅ Colunas disponíveis:", df_expanded.columns.tolist())
print("✅ Dados carregados e normalizados!")

# 🔧 Transformação

In [ ]:
# Corrigir tipos numéricos
df_expanded["charges_total"] = pd.to_numeric(df_expanded["charges_total"], errors="coerce").fillna(0)
df_expanded["tenure"] = pd.to_numeric(df_expanded["tenure"], errors="coerce").fillna(0).astype(int)

# Criar Ticket Médio
df_expanded["ticket_medio"] = df_expanded.apply(lambda row: row.charges_total / row.tenure if row.tenure > 0 else 0, axis=1)

# Criar Faixa de Tempo de Cliente
bins = [0, 6, 12, 24, 36, 48, 60, df_expanded["tenure"].max()]
labels = ["0-6m", "7-12m", "13-24m", "25-36m", "37-48m", "49-60m", "60m+"]
df_expanded["faixa_tempo_cliente"] = pd.cut(df_expanded["tenure"], bins=bins, labels=labels)

# 📊 Carga e Análise

In [ ]:
sns.set(style="whitegrid")

# Taxa de churn
churn_rate = df_expanded["churn"].value_counts(normalize=True) * 100
print("\n📊 Taxa de Churn (%):")
print(churn_rate)

# Gráfico: Churn por tipo de contrato
plt.figure(figsize=(6,4))
sns.countplot(data=df_expanded, x="contract", hue="churn", palette="Set2")
plt.title("Churn por Tipo de Contrato")
plt.show()

# Gráfico: Distribuição do tempo de cliente
plt.figure(figsize=(6,4))
sns.histplot(data=df_expanded, x="tenure", hue="churn", multiple="stack", palette="coolwarm", bins=20)
plt.title("Distribuição do Tempo de Cliente x Churn")
plt.show()

# Gráfico: Ticket médio x Churn
plt.figure(figsize=(6,4))
sns.boxplot(data=df_expanded, x="churn", y="ticket_medio", palette="pastel")
plt.title("Ticket Médio x Churn")
plt.show()

# 📄 Relatório Final

In [ ]:
print("\n💡 INSIGHTS ESTRATÉGICOS:")
print(f"- Taxa de churn total: {churn_rate.get('Yes', 0):.2f}%")
print("- Clientes com contrato mensal apresentam maior risco de evasão.")
print("- Grande parte dos cancelamentos ocorre até 12 meses de contrato.")
print("- Ticket médio mais alto está relacionado a maior chance de churn.")
print("- 'Preço mais alto' é o motivo de cancelamento mais recorrente.")

# Análise de Correlação
df_corr = df_expanded.copy()
df_corr['churn_num'] = df_corr['churn'].map({'Yes': 1, 'No': 0})
df_corr_numeric = df_corr.select_dtypes(include=['int64', 'float64'])

plt.figure(figsize=(10,6))
sns.heatmap(df_corr_numeric.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title("📈 Matriz de Correlação entre Variáveis Numéricas")
plt.show()